## Affordability Calculator Inputs

The `AffordabilityCalculator` requires the following inputs for its `calculate_affordability` method.

1.  **`predicted_price` (float or int):**
    * The predicted price of the property.
    * Example: `500000`

2.  **`buyer_profile` (dictionary):**
    * Information about the buyer(s).
    * Required keys:
        * `is_first_timer` (boolean): `True` if first-time buyer, `False` otherwise.
        * `monthly_household_income` (float or int): Combined gross monthly income.
        * `is_married` (boolean): `True` if married, `False` otherwise.
        * `is_near_parents` (boolean): `True` if property is near parents, `False` otherwise.
        * `is_extended_family` (boolean): `True` if property is near extended family, `False` otherwise.
        * `monthly_debt_obligations` (float or int, optional): Total monthly debt obligations (defaults to 0 if not provided).
        * `cash_on_hand` (float or int): Amount of liquid cash the buyer has on hand.
    * Example:
        ```python
        buyer_profile = {
            "is_first_timer": True,
            "monthly_household_income": 6000,
            "is_married": True,
            "is_near_parents": True,
            "is_extended_family": False,
            "monthly_debt_obligations": 500,
            "cash_on_hand": 50000
        }
        ```

3.  **`loan_options` (list of dictionaries):**
    * A list of dictionaries representing different loan scenarios.
    * Each dictionary must have the following keys:
        * `name` (string): Descriptive name of the loan option.
        * `interest_rate` (float): Annual interest rate.
        * `loan_tenure_years` (int): Loan term in years.
    * Example:
        ```python
        loan_options = [
            {
                "name": "HDB Loan (25 years)",
                "interest_rate": 2.6,
                "loan_tenure_years": 25
            },
            {
                "name": "Bank Loan (25 years)",
                "interest_rate": 1.8,
                "loan_tenure_years": 25
            }
        ]
        ```

4.  **`cpf_oa_balance` (float or int):**
    * The buyer's CPF Ordinary Account (OA) balance.
    * Example: `100000`

5.  **`cash_on_hand` (float or int):**
    * The buyer's available cash on hand.
    * Example: `50000`

## Affordability Calculator Outputs

The `AffordabilityCalculator`'s `calculate_affordability` method returns a dictionary containing various financial analyses. Here's a breakdown of the output structure:

```python
{
    "upfront_costs": {
        "downpayment": float,
        "bsd": float,
        "legal_fees": float,
        "valuation_fee": float,
        "option_fee": float,
        "total": float
    },
    "grants": {
        "enhanced_housing_grant": float,
        "family_grant": float,
        "proximity_housing_grant": float,
        "total": float
    },
    "cpf_analysis": {
        "cpf_for_downpayment": float,
        "cpf_for_other_fees": float,
        "total_cpf_used": float,
        "total_cash_required": float,
        "remaining_cpf_balance": float
    },
    "loan_scenarios": {
        "loan_option_name_1": {
            "loan_details": {
                "loan_amount": float,
                "monthly_payment": float,
                "total_interest_paid": float,
                "total_repayment": float
            },
            "affordability": {
                "msr": float,
                "msr_status": str,
                "tdsr": float,
                "tdsr_status": str,
                "cash_flow_after_mortgage": float,
                "months_of_reserves": float,
                "overall_assessment": str
            }
        },
        "loan_option_name_2": {
            # ... (same structure as above for each loan option)
        },
        # ... (more loan options)
    }
}

In [2]:
import json

In [3]:
class AffordabilityCalculator:
    def __init__(self):
        pass
    
    def calculate_upfront_costs(self, predicted_price):
        """Calculate all one-time costs associated with purchasing."""
        # Downpayment calculation (25% for HDB loans)
        downpayment_percentage = 0.25
        downpayment = predicted_price * downpayment_percentage
        
        # Buyer's Stamp Duty (BSD)
        if predicted_price <= 180000:
            bsd = predicted_price * 0.01
        elif predicted_price <= 360000:
            bsd = 1800 + (predicted_price - 180000) * 0.02
        else:
            bsd = 5400 + (predicted_price - 360000) * 0.03
        
        # Legal fees (approximate)
        legal_fees = 2500
        
        # Valuation fee (approximate)
        valuation_fee = 500
        
        # Option fee
        option_fee = 1000 if predicted_price < 500000 else 2000
        
        total = downpayment + bsd + legal_fees + valuation_fee + option_fee
        
        return {
            "downpayment": downpayment,
            "bsd": bsd,
            "legal_fees": legal_fees,
            "valuation_fee": valuation_fee,
            "option_fee": option_fee,
            "total": total
        }
    
    def calculate_eligible_grants(self, buyer_profile, predicted_price):
        """Calculate eligible housing grants based on buyer profile."""
        grants = {
            "enhanced_housing_grant": 0,
            "family_grant": 0,
            "proximity_housing_grant": 0,
            "total": 0
        }
        
        # Enhanced CPF Housing Grant (EHG)
        if buyer_profile.get("is_first_timer", False):
            income = buyer_profile.get("monthly_household_income", 0)
            
            if income <= 1500:
                grants["enhanced_housing_grant"] = 80000
            elif income <= 9000:
                # EHG decreases by $5,000 for each $500 increase in income
                income_bracket = (income - 1500) // 500
                if (income - 1500) % 500 > 0:
                    income_bracket += 1
                grants["enhanced_housing_grant"] = max(80000 - (income_bracket * 5000), 0)
        
        # Family Grant
        if buyer_profile.get("is_first_timer", False) and buyer_profile.get("is_married", False):
            grants["family_grant"] = 50000 if predicted_price <= 550000 else 40000
        
        # Proximity Housing Grant
        if buyer_profile.get("is_near_parents", False):
            grants["proximity_housing_grant"] = 30000
        elif buyer_profile.get("is_extended_family", False):
            grants["proximity_housing_grant"] = 15000
        
        grants["total"] = (
            grants["enhanced_housing_grant"] + 
            grants["family_grant"] + 
            grants["proximity_housing_grant"]
        )
        
        return grants
    
    def calculate_loan(self, predicted_price, downpayment, grants, loan_details):
        """Calculate loan details including monthly payments."""
        # Calculate loan amount
        loan_amount = predicted_price - downpayment - grants["total"]
        
        # Monthly installment calculation
        monthly_interest_rate = loan_details["interest_rate"] / 12 / 100
        loan_term_months = loan_details["loan_tenure_years"] * 12
        
        # Using the formula: PMT = P * r * (1 + r)^n / ((1 + r)^n - 1)
        numerator = loan_amount * monthly_interest_rate * pow(1 + monthly_interest_rate, loan_term_months)
        denominator = pow(1 + monthly_interest_rate, loan_term_months) - 1
        monthly_payment = numerator / denominator
        
        total_repayment = monthly_payment * loan_term_months
        
        return {
            "loan_amount": loan_amount,
            "monthly_payment": monthly_payment,
            "total_interest_paid": total_repayment - loan_amount,
            "total_repayment": total_repayment
        }
    
    def analyze_cpf_usage(self, upfront_costs, cpf_oa_balance, buyer_profile):
        """Analyze how CPF can be used for housing payments."""
        # How much of downpayment can be paid using CPF
        max_cpf_for_downpayment = min(upfront_costs["downpayment"], cpf_oa_balance)
        
        # Remaining cash needed for downpayment
        cash_for_downpayment = upfront_costs["downpayment"] - max_cpf_for_downpayment
        
        # Other fees that can potentially use CPF
        eligible_cpf_expenses = upfront_costs["legal_fees"] + upfront_costs["bsd"]
        remaining_cpf_after_downpayment = cpf_oa_balance - max_cpf_for_downpayment
        max_cpf_for_other_fees = min(eligible_cpf_expenses, remaining_cpf_after_downpayment)
        
        # Cash required for other fees
        cash_for_other_fees = eligible_cpf_expenses - max_cpf_for_other_fees
        
        # Cash needed for valuation and option fee (cannot use CPF)
        mandatory_cash_expenses = upfront_costs["valuation_fee"] + upfront_costs["option_fee"]
        
        return {
            "cpf_for_downpayment": max_cpf_for_downpayment,
            "cpf_for_other_fees": max_cpf_for_other_fees,
            "total_cpf_used": max_cpf_for_downpayment + max_cpf_for_other_fees,
            "total_cash_required": cash_for_downpayment + cash_for_other_fees + mandatory_cash_expenses,
            "remaining_cpf_balance": cpf_oa_balance - max_cpf_for_downpayment - max_cpf_for_other_fees
        }
    
    def assess_affordability(self, loan_details, buyer_profile, cpf_analysis):
        """Assess if the property is affordable based on financial metrics."""
        # Mortgage Servicing Ratio (MSR) calculation - for HDB
        # MSR should not exceed 30% of gross monthly income
        msr = loan_details["monthly_payment"] / buyer_profile["monthly_household_income"]
        
        # Total Debt Servicing Ratio (TDSR) calculation
        # TDSR should not exceed 60% of gross monthly income (includes all debts)
        tdsr = (loan_details["monthly_payment"] + buyer_profile.get("monthly_debt_obligations", 0)) / buyer_profile["monthly_household_income"]
        
        # Cash flow assessment
        monthly_net_income = buyer_profile["monthly_household_income"] - buyer_profile.get("monthly_debt_obligations", 0)
        cash_flow_after_mortgage = monthly_net_income - loan_details["monthly_payment"]
        
        # Cash reserves assessment
        months_of_reserves = buyer_profile.get("cash_on_hand", 0) / loan_details["monthly_payment"] if loan_details["monthly_payment"] > 0 else 0
        
        # Determine overall affordability
        overall_assessment = self.determine_overall_affordability(msr, tdsr, cash_flow_after_mortgage, months_of_reserves)
        
        return {
            "msr": msr,
            "msr_status": "Within limits" if msr <= 0.3 else "Exceeds 30% MSR limit",
            "tdsr": tdsr,
            "tdsr_status": "Within limits" if tdsr <= 0.6 else "Exceeds 60% TDSR limit",
            "cash_flow_after_mortgage": cash_flow_after_mortgage,
            "months_of_reserves": months_of_reserves,
            "overall_assessment": overall_assessment
        }
    
    def determine_overall_affordability(self, msr, tdsr, cash_flow_after_mortgage, months_of_reserves):
        """Determine overall affordability based on multiple factors."""
        if msr > 0.3 or tdsr > 0.6:
            return "Not affordable: Loan ratios exceed regulatory limits"
        
        if cash_flow_after_mortgage < 1000:
            return "Caution: Cash flow buffer is tight after mortgage payment"
        
        if months_of_reserves < 6:
            return "Caution: Limited cash reserves for emergencies"
        
        if msr > 0.25 or tdsr > 0.5:
            return "Moderate strain: Housing costs are significant portion of income"
        
        return "Affordable: Housing costs are within reasonable limits"
    
    def calculate_affordability(self, predicted_price, buyer_profile, loan_options, cpf_oa_balance, cash_on_hand):
        """Main calculator function to calculate overall affordability."""
        results = {}
        
        # Update buyer profile with cash on hand
        buyer_profile["cash_on_hand"] = cash_on_hand
        
        # Calculate upfront costs
        upfront_costs = self.calculate_upfront_costs(predicted_price)
        results["upfront_costs"] = upfront_costs
        
        # Calculate eligible grants
        grants = self.calculate_eligible_grants(buyer_profile, predicted_price)
        results["grants"] = grants
        
        # Calculate CPF usage
        cpf_analysis = self.analyze_cpf_usage(upfront_costs, cpf_oa_balance, buyer_profile)
        results["cpf_analysis"] = cpf_analysis
        
        # Calculate different loan scenarios
        results["loan_scenarios"] = {}
        
        for loan_option in loan_options:
            loan_details = self.calculate_loan(
                predicted_price, 
                upfront_costs["downpayment"], 
                grants, 
                loan_option
            )
            
            affordability = self.assess_affordability(loan_details, buyer_profile, cpf_analysis)
            
            results["loan_scenarios"][loan_option["name"]] = {
                "loan_details": loan_details,
                "affordability": affordability
            }
        
        return results


def example_usage():
    def predict_price(location, flat_type, floor_area, floor, age):
        # Our existing model logic here
        return 500000  # Example predicted price
    
    # Get a predicted price from your model
    predicted_price = predict_price(
        location="Tampines",
        flat_type="4-Room", 
        floor_area=90, 
        floor=12, 
        age=5
    )
    
    # Create buyer profile
    buyer_profile = {
        "is_first_timer": True,
        "monthly_household_income": 6000,
        "is_married": True,
        "is_near_parents": True,
        "is_extended_family": False,
        "monthly_debt_obligations": 500
    }
    
    # Loan options to compare
    loan_options = [
        {
            "name": "HDB Loan (25 years)",
            "interest_rate": 2.6,
            "loan_tenure_years": 25
        },
        {
            "name": "Bank Loan (25 years)",
            "interest_rate": 1.8,
            "loan_tenure_years": 25
        },
        {
            "name": "Bank Loan (30 years)",
            "interest_rate": 1.9,
            "loan_tenure_years": 30
        }
    ]
    
    # CPF OA balance and cash on hand
    cpf_oa_balance = 100000
    cash_on_hand = 50000
    
    # Calculate affordability
    calculator = AffordabilityCalculator()
    results = calculator.calculate_affordability(
        predicted_price,
        buyer_profile,
        loan_options,
        cpf_oa_balance,
        cash_on_hand
    )
    
    return results

if __name__ == "__main__":
    # For testing purposes
    results = example_usage()
    import json
    print(json.dumps(results, indent=2))

{
  "upfront_costs": {
    "downpayment": 125000.0,
    "bsd": 9600.0,
    "legal_fees": 2500,
    "valuation_fee": 500,
    "option_fee": 2000,
    "total": 139600.0
  },
  "grants": {
    "enhanced_housing_grant": 35000,
    "family_grant": 50000,
    "proximity_housing_grant": 30000,
    "total": 115000
  },
  "cpf_analysis": {
    "cpf_for_downpayment": 100000,
    "cpf_for_other_fees": 0,
    "total_cpf_used": 100000,
    "total_cash_required": 39600.0,
    "remaining_cpf_balance": 0
  },
  "loan_scenarios": {
    "HDB Loan (25 years)": {
      "loan_details": {
        "loan_amount": 260000.0,
        "monthly_payment": 1179.5407187112553,
        "total_interest_paid": 93862.21561337658,
        "total_repayment": 353862.2156133766
      },
      "affordability": {
        "msr": 0.1965901197852092,
        "msr_status": "Within limits",
        "tdsr": 0.27992345311854255,
        "tdsr_status": "Within limits",
        "cash_flow_after_mortgage": 4320.459281288745,
        "mo